# Get all clubs type et manufacturers

In [96]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def write_to_json_file(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    print("Data has been written to", file_path)

def add_ids_and_save(data, filename):
    data_with_ids = [{"id": idx + 1, "name": item} for idx, item in enumerate(data)]
    write_to_json_file(filename, data_with_ids)

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get("https://www.usga.org/InfoClubsDB/Search.aspx")
wait = WebDriverWait(driver, 10)
select_element = wait.until(EC.presence_of_element_located((By.ID, 'ddlclubtypes')))
select_club_type = Select(select_element)

club_type_values = [option.get_attribute('value') for option in select_club_type.options
                    if option.get_attribute('value') and option.get_attribute('value') != "All" and option.get_attribute('value').strip()]

all_manufacturer_values = set()
for club_type_value in club_type_values:
    select_club_type = Select(wait.until(EC.element_to_be_clickable((By.ID, 'ddlclubtypes'))))
    select_club_type.select_by_value(club_type_value)
    time.sleep(2)

    select_manufacturer = Select(wait.until(EC.element_to_be_clickable((By.ID, 'ddlManf'))))
    manufacturer_values = [option.get_attribute('value') for option in select_manufacturer.options
                           if option.get_attribute('value') and option.get_attribute('value') != "All" and option.get_attribute('value').strip()]
    all_manufacturer_values.update(manufacturer_values)

driver.quit()

add_ids_and_save(club_type_values, 'club_types.json')
add_ids_and_save(list(all_manufacturer_values), 'club_manufacturers.json')


Data has been written to club_types.json
Data has been written to clubs_manufacturers.json


# Get all clubs infos

In [99]:
import json
import requests
from bs4 import BeautifulSoup
import urllib.parse

def load_club_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

global_club_id = 1

def extract_club_info(html_content, club_type_id, manufacturer_id, global_id):
    soup = BeautifulSoup(html_content, 'html.parser')
    clubs = []
    containers = soup.find_all(id='conformingContainer')
    for container in containers:
        club = {
            "id": global_id,
            "club_type_id": club_type_id, 
            "manufacturer_id": manufacturer_id
        }
        global_id += 1  
        rows = container.find_all(class_='conformRow')
        for row in rows:
            label_element = row.find(class_='lblLong')
            info_element = row.find(class_='conformInfo')
            if label_element and info_element:
                label = label_element.text.strip().replace(" ", "_").lower()
                if label == "product_name":
                    club['product_name'] = info_element.text.strip()
                elif label == "loft":
                    club['loft'] = info_element.text.strip()
                elif label == "club_#":
                    club['club_sign'] = info_element.text.strip()
                elif label == "markings":
                    club['markings'] = info_element.text.strip()
                elif label == "image":
                    image_container = container.find(id=lambda x: x and x.startswith('cRowImage'))
                    if image_container:
                        image_link = image_container.find('img')
                        club['image'] = image_link['src'] if image_link else None
                    else:
                        club['image'] = None
                elif label in ["meets_pre-2010_rules_of_golf", "meets_2010_groove_rules"]:
                    club[label] = info_element.text.strip() == "Yes"
        clubs.append(club)
    return clubs, global_id

club_types_data = load_club_data('club_types.json')
manufacturers_data = load_club_data('clubs_manufacturers.json')

all_clubs = []
global_club_id = 1  
for club_type in club_types_data:
    for manufacturer in manufacturers_data:  
        url = f"https://www.usga.org/InfoClubsDB/ResultDisplay.aspx?clubtype={urllib.parse.quote(club_type['name'])}&Manf={urllib.parse.quote(manufacturer['name'])}&Prod=All"
        response = requests.get(url)
        if response.status_code == 200:
            new_clubs, global_club_id = extract_club_info(response.content, club_type['id'], manufacturer['id'], global_club_id)
            all_clubs.extend(new_clubs)

with open('clubs.json', 'w', encoding='utf-8') as file:
    json.dump(all_clubs, file, indent=4, ensure_ascii=False)

print("Extraction complete. Data written to 'clubs.json'.")


Test extraction complete. Data written to 'clubs.json'.


# Generate SQL Query

In [25]:
import json
import uuid

def read_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def write_sql_file(sql_commands, file_name='update_clubs.sql'):
    with open(file_name, 'w') as file:
        for command in sql_commands:
            file.write(f"{command}\n")

def sanitize_input(input_string):
    return input_string.replace("'", "''").replace('View Markings\n\n', '').replace('\n', " ").replace('\r', " ")

def clean(input_string):
    if not input_string:
        return "\'\'"

def main():
    clubs = read_json('clubs.json')
    club_types = read_json('club_types.json')
    club_manufacturers = read_json('club_manufacturers.json')
    
    sql_commands = []
    
    manufacturer_ids = {}
    for manufacturer in club_manufacturers:
        manufacturer_id = manufacturer['id']
        manufacturer_ids[manufacturer['id']] = manufacturer_id
        sql_commands.append(f"INSERT INTO club_manufacturers (id, manufacturer_name) VALUES ('{manufacturer_id}', '{sanitize_input(manufacturer['name'])}') ON CONFLICT (id) DO UPDATE SET manufacturer_name = EXCLUDED.manufacturer_name;")
    
    type_ids = {}
    for type in club_types:
        type_id = type['id']
        type_ids[type['id']] = type_id
        sql_commands.append(f"INSERT INTO club_types (id, club_type_name) VALUES (\'{type_id}\', \'{type['name']}\') ON CONFLICT (id) DO UPDATE SET club_type_name = EXCLUDED.club_type_name;")
    
    for club in clubs:
        club_id = club.get('id')
        product_name = sanitize_input(club.get('product_name', ''))
        loft = sanitize_input(club.get('loft', ''))
        markings = sanitize_input(club.get('markings', ''))
        club_sign = sanitize_input(club.get('club_sign', ''))
        image = club.get('image', '')
        meets_pre_2010 = True if not club.get('meets_pre-2010_rules_of_golf', '') else club.get('meets_pre-2010_rules_of_golf', '')
        meets_2010_groove = True if not club.get('meets_2010_groove_rules', '') else club.get('meets_2010_groove_rules', '')
        type_id = type_ids.get(club['club_type_id'])
        manufacturer_id = manufacturer_ids.get(club['manufacturer_id'])
        
        sql_commands.append(
            f"""INSERT INTO clubs (id, product_name, club_type_id, manufacturer_id, loft, markings, club_sign, meets_pre_2010_rules_of_golf, meets_2010_grooveRules, image) 
            VALUES ('{club_id}', '{product_name}', '{type_id}', '{manufacturer_id}', '{loft}', '{markings}', '{club_sign}', {meets_pre_2010}, {meets_2010_groove}, '{image}') ON CONFLICT (id) DO UPDATE SET product_name = EXCLUDED.product_name, club_type_id = EXCLUDED.club_type_id, manufacturer_id = EXCLUDED.manufacturer_id, loft = EXCLUDED.loft, markings = EXCLUDED.markings, club_sign = EXCLUDED.club_sign, meets_pre_2010_rules_of_golf = EXCLUDED.meets_pre_2010_rules_of_golf, meets_2010_grooveRules = EXCLUDED.meets_2010_grooveRules, image = EXCLUDED.image;"""
        )
    
    write_sql_file(sql_commands)
    print("Query written to update_clubs.sql")


if __name__ == "__main__":
    main()
